In [120]:
import requests as req
import pandas as pd
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import datetime
from statsmodels.tools import add_constant
from sklearn.linear_model import LinearRegression 
%matplotlib inline
import sklearn
from sklearn.decomposition import PCA

In [121]:
price_data =  pd.read_hdf(r'C:\Users\Anonymous\Documents\Data Analysis\Backtesting - Final\Final\Backtesting\FullOHLCFinal11.h5')  

In [122]:
close_df = price_data
close_df = close_df.drop(columns=['Open', 'High', 'Low', 'Volume', 'VolumeCcy', 'Returns', 'Log Returns'])

In [123]:
returns_df = (close_df.groupby(['Timestamp', 'Token'])['Close']
    .first()
    .unstack())

returns_df = returns_df.sort_index(ascending=True)
returns_df = returns_df.pct_change(1)
returns_df = returns_df.drop(columns=['TUSDUSDT.BINANCE', 'USTUSDT.BINANCE', 'USDCUSDT.BINANCE', 'UTKUSDT.BINANCE', 'DAIUSDT.BINANCE'])

In [124]:
recent_returns = returns_df.tail(60*24*180)

In [125]:
snap_returns = recent_returns.tail(60*24*5)
returns_df = snap_returns

In [126]:
standardized = (returns_df - returns_df.mean()) / returns_df.std()

In [127]:
estimation_window = (60/252) * len(returns_df)

reversion_t = len(returns_df) / estimation_window * 0.25

In [128]:
covariance_df = standardized.cov()

In [129]:
pca = PCA()
pca.fit(covariance_df)

PCA()

In [130]:
total_var_explained = pd.Series(pca.explained_variance_ratio_).to_frame('Explained Variance').head(5).style.format('{:,.2%}'.format)
total_var_explained

,Explained Variance
0,86.07%
1,0.52%
2,0.21%
3,0.13%
4,0.12%


In [131]:
factors = 10

eigvec =  pca.components_[:factors]

eigval = pca.explained_variance_[:factors]

In [132]:
N = len (standardized)
Std = returns_df.std()


In [133]:
Q = np.zeros((factors, N))
returns_df = returns_df.fillna(0)

In [134]:
# eigenportfolios
# long/short pairs on the level of sectors
# first eigenportfolio is market portfolio, rest are orthogonal
# Q is the amount invested in each stock based on each eigenportfolio

for p in range(factors):
    for i in range(len(covariance_df)):
        Q[p, i] = np.real(np.divide(eigvec[p, i], Std[i]))
        


In [135]:
# p x T
# Market factors returns 
# Given the eigenportfolio and amounts (Q) we can calculate F, the returns of each eigneportfolio
F = np.inner(Q, returns_df.T)
    
# Market factor coefficients (betas)
# Multivariate least squares of R on F 
# Regress R on F so that we can calculate the coefficients, and get U the residual, which represents idiosyncratic returns of each stock
model = LinearRegression()
model.fit(F.T, returns_df.T)

# N x p 
# factor loadings
L = model.coef_

# Calculate residuals according to R = LF + U     
# U is returns not explained by the eigenportfolios. So, we can model this using an OU process to find a mean reverting component to trade
U = returns_df - np.matmul(L, F)

In [136]:
U

Token,1INCHUSDT.BINANCE,AAVEUSDT.BINANCE,ABTUSDT.BINANCE,ACAUSDT.BINANCE,ACTUSDT.BINANCE,ADAUSDT.BINANCE,AERGOUSDT.BINANCE,AEUSDT.BINANCE,AGLDUSDT.BINANCE,AKITAUSDT.BINANCE,...,YGGUSDT.BINANCE,YOUUSDT.BINANCE,YOYOUSDT.BINANCE,ZBCUSDT.BINANCE,ZECUSDT.BINANCE,ZENUSDT.BINANCE,ZILUSDT.BINANCE,ZKSUSDT.BINANCE,ZRXUSDT.BINANCE,ZYROUSDT.BINANCE
Timestamp,,,,,,,,,,,,,,,,,,,,,
1648594920000,0.000281,-0.001971,0.000201,-0.000691,0.001542,0.000174,0.000127,-0.001827,-0.000651,0.000996,...,-0.000808,0.000238,-0.000464,0.000372,-0.000039,0.000405,0.001420,0.002720,0.001310,0.012333
1648594980000,-0.000306,0.001365,-0.000944,0.000019,0.002376,0.000260,-0.001598,-0.000399,-0.000366,-0.001193,...,-0.000620,-0.000711,-0.000305,-0.000078,-0.000343,-0.000130,0.004446,0.000458,0.000259,-0.005687
1648595040000,0.000388,-0.000274,-0.000593,-0.000366,-0.000125,-0.001022,-0.001938,0.003291,-0.001238,-0.000519,...,0.002147,-0.001217,0.008186,-0.001743,0.000008,-0.001014,0.003594,0.000386,-0.000496,-0.019070
1648595100000,-0.000307,-0.000658,-0.000157,0.000145,-0.003138,-0.000735,0.002095,0.002074,-0.001141,-0.000083,...,-0.000179,0.001280,-0.001568,0.002426,-0.001063,0.000525,-0.001955,-0.000543,-0.001591,0.013763
1648595160000,0.000185,-0.000152,-0.001456,0.000208,-0.000381,-0.000303,-0.000298,-0.000601,0.000837,-0.001405,...,0.000412,-0.000138,-0.004853,0.000963,-0.001472,0.000763,-0.000303,0.000147,0.000098,0.004688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649026620000,-0.002569,-0.000430,-0.000372,-0.000797,-0.003435,0.000127,-0.002622,0.002361,-0.001186,0.003168,...,-0.000428,-0.001039,-0.000297,-0.005554,-0.001458,-0.000684,-0.002874,-0.002572,-0.000438,0.000057
1649026680000,-0.000804,0.000192,-0.001988,0.000582,0.001811,0.000190,-0.001295,-0.000755,-0.001540,-0.000252,...,-0.000741,-0.000019,0.000059,-0.000677,0.000126,-0.000056,-0.001280,-0.000237,0.003032,-0.000437
1649026740000,-0.001747,-0.002006,-0.000124,0.000730,-0.000444,-0.000329,0.001432,-0.000673,-0.000658,0.001333,...,0.001354,0.000924,-0.000076,-0.001042,-0.000284,-0.001396,0.001744,0.000621,-0.001036,-0.001391


In [137]:
# residual_arr = U.fillna(0)

In [138]:
residual_arr = U.to_numpy()

In [139]:
residual_arr = residual_arr.transpose()

In [140]:
stock_score = []  
stock_kappa = [] 
beta_list = []
low_beta = []
arr_beta = []

In [141]:
for i in range(residual_arr.shape[0]):
        X = np.cumsum(residual_arr[i])
        t = len(X) - 1
        Xt = X[0:-1].reshape(t,1)
        Xt1 = X[1:].reshape(t,1)
        ls = LinearRegression(fit_intercept=True).fit(Xt, Xt1)
        beta = ls.coef_[0][0]
        arr_beta.append(beta)
        if beta < 0.96:
            low_beta.append(beta)
        if beta >= 1 :
            beta = .9672 
            
        alpha = ls.intercept_[0]
        # epsilon -> random process of t+1 residual  
        epsilon = Xt1 - alpha - beta * Xt
        k = -np.log10(beta) * len(returns_df)
        m = alpha / (1 - beta)
        sigma = np.sqrt((np.var(epsilon))/(1-np.power(beta, 2)))
        score = (X[t]-m) / sigma 
        # print(f'score {score}, {i}')
        # adjust score for drift 
        modscore = (score - (alpha/(k*sigma)))
        stock_kappa.append(k)
        stock_score.append(modscore)

In [142]:
# filter out stocks based on mean reversion factor
# estimated mean reversion must occur within ~8 days
filtered = [i for i,x in enumerate(stock_kappa) if (x < reversion_t)]

In [148]:
filtered

[0,
 12,
 14,
 15,
 22,
 24,
 29,
 44,
 50,
 51,
 55,
 64,
 65,
 69,
 92,
 112,
 113,
 118,
 130,
 131,
 141,
 153,
 161,
 162,
 169,
 179,
 197,
 200,
 203,
 216,
 236,
 244,
 252,
 255,
 260,
 264,
 266,
 268,
 279,
 291,
 327,
 336]